In [8]:
import numpy as np
%matplotlib inline 
import matplotlib.pyplot as plt

class DifferentiationandIntegration:
    def __init__(self,f,delx = 1e-6):
        self.f = f
        self.delx = delx
    def forward_difference(self,x):
            """Numerical derivative using forward difference."""
            return (self.f(x + self.delx) - self.f(x)) / self.delx

    def backward_difference(self, x):
            """Numerical derivative using backward difference."""
            return (self.f(x) - self.f(x - self.delx)) / self.delx

    def central_difference(self, x):
            """Numerical derivative using central difference ."""
            return (self.f(x + self.delx) - self.f(x - self.delx)) / (2 * self.delx)
    def MidpointMethod(self,x1,x2):
        arr = np.arange(x1, x2, self.delx)
        midpoints = arr + self.delx / 2
        fx = self.f(midpoints)
        area = np.sum(fx) * self.delx
        return area
    
    def TrapezoidalMethod(self, x1, x2):
        arr = np.arange(x1, x2 + self.delx, self.delx)
        fx = self.f(arr)
        area = (fx[0] + fx[-1]) / 2 + np.sum(fx[1:-1])
        area *= self.delx
        return area

In [9]:
f = lambda x : x**3 + 2*x**2 - 3
obj = DifferentiationandIntegration(f)

print( obj.MidpointMethod(2,3))
obj.TrapezoidalMethod(2,3)

25.91666666891397


np.float64(25.916708668935847)

In [10]:
class SimpleHarmonicOscillator:
    def __init__(self,x0,v0,n,dt,k,m):
        self.x0 = x0
        self.v0 = v0
        self.n = n
        self.dt =dt
        self.k = k
        self.m = m
        self.t = np.arange(0,n*dt,dt)
        
    def EulerMethod(self):
        x = np.zeros(self.n,dtype=float)
        v = np.zeros(self.n,dtype=float)
        x[0] = self.x0
        v[0] = self.v0
        for i in range(1,self.n):
            x[i] = x[i-1] + v[i-1]*self.dt
            v[i] = v[i-1] + (-self.k/self.m)*x[i-1]*self.dt
        return x,v
    def ModifiedEuler(self):
        x = np.zeros(self.n,dtype=float)
        v = np.zeros(self.n,dtype=float)
        x[0] = self.x0
        v[0] = self.v0
        for i in range(1,self.n):
            v[i] = v[i-1] + (-self.k/self.m)*x[i-1]*self.dt
            x[i] = x[i-1] + v[i]*self.dt
        return x,v
    def Verlet(self):
        x = np.zeros(self.n,dtype=float)
        v = np.zeros(self.n,dtype=float)
        x[0] = self.x0
        x[1] = v[0]*self.dt + (-self.k/self.m)*x[0]*self.dt**2
        v[0] = self.v0
        for i in range(2,self.n+1):
            x[i] = 2*x[i-1] + (-self.k/self.m)*x[i-1]*self.dt**2 - x[i-2]
            v[i-1] = (x[i] - x[i-2])/(2*self.dt)
        x = x[:-1]
        return x,v
    def errorEuler(self):
        error_M = np.array([[1,self.dt],[-(self.k/self.m)*self.dt,1]])
        eig_value,eig_vector = np.linalg.eig(error_M)
        return eig_value
    def errorModifiedEuler(self):
        error_M = np.array([[(1-(self.k*self.dt**2)/self.m),self.dt],[-self.k*self.dt/self.m,1]])
        eig_value,eig_vector = np.linalg.eig(error_M)
        return eig_value
    def PotentialEnergy(self,x):
        return 0.5*self.k*x**2
    def KineticEnergy(self,v):
        return 0.5*self.m*v**2


In [11]:
n = 10000
dt = 0.01
x0 = 0
v0 = 1
k = 1
m = 1
SHO = SimpleHarmonicOscillator(x0,v0=v0,n=n,dt=dt,k=k,m=m)
x,v = SHO.EulerMethod()
x

array([ 0.        ,  0.01      ,  0.02      , ..., -0.88161808,
       -0.86768941, -0.85367257], shape=(10000,))

In [12]:
def analytical_sol(t,k,m,x0,v0):
    return x0*np.cos(np.sqrt(k/m)*t)+(v0/np.sqrt(k/m))*np.sin(np.sqrt(k/m)*t)

def analytical_vel(t,k,m,x0,v0):
    return -x0*np.sqrt(k/m)*np.sin(np.sqrt(k/m)*t)+(v0)*np.cos(np.sqrt(k/m)*t)

t = np.arange(0,n*dt,dt)
x_analytical = analytical_sol(t,k,m,x0,v0)
v_analytical = analytical_vel(t,k,m,x0,v0)

In [13]:
error = SHO.errorEuler()